In [ ]:
%load_ext sql

In [19]:
%sql duckdb:///data/local.duckdb


Connecting and switching to connection 'duckdb:///data/local.duckdb'

In [20]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;


Running query in 'duckdb:///data/local.duckdb'

Success


In [21]:
%%sql

CREATE TABLE point_data AS
SELECT *
FROM ST_Read("./data/osm21_cemetery.geojson")

Running query in 'duckdb:///data/local.duckdb'

Count


In [22]:
%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH centroid AS (
    FROM point_data
    SELECT centroid: geom.ST_Centroid(),
)
FROM centroid
SELECT
    id: ROW_NUMBER() OVER (ORDER BY centroid),
    locationX: centroid.ST_X(),
    locationy: centroid.ST_Y(),
    description: substr(md5(random()::VARCHAR), 1, 10)


Running query in 'duckdb:///data/local.duckdb'

Count


In [23]:
%sql SELECT * FROM point_data_centroid

Running query in 'duckdb:///data/local.duckdb'

id,locationX,locationy,description
1,71.60207635350525,54.78643014239917,b6cf96d44f
2,68.07547055464238,40.23959944486414,c476c8cb3f
3,22.71972581720911,48.451325419849496,cc7ff5fbca
4,29.491294654319063,46.07657549119183,1ce286ab90
5,79.38921734504402,52.74566659600047,bded6d885f
6,6.359217615158059,43.142364551113644,5f2e50165d
7,1.9459106666618595,49.15726517327247,0e5b974d0b
8,7.758160430065118,43.808794503076086,7b71bdc0b5
9,-0.9758063970803335,51.474531240130844,56921ad21e
10,66.95163331926233,39.698323306344484,d015471104


In [24]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb:///data/local.duckdb'

count_star()
402325


In [25]:
%%sql
SUMMARIZE point_data_centroid

Running query in 'duckdb:///data/local.duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,BIGINT,1,402325,377825,201163.0,116141.3678633357,100582,201163,301744,402325,0.00
locationX,DOUBLE,-179.12087778314643,179.80886915,326174,1.0731648788767603,60.20403126591059,-45.852936818367894,9.42341979115554,28.65024833709391,402325,0.00
locationy,DOUBLE,-66.52681067468725,78.92123011264881,477667,38.357988604940694,20.33379941938753,36.85145636350717,44.581229586115164,49.950301679313824,402325,0.00
description,VARCHAR,00000e9bf0,ffffd882fb,350550,None,None,None,None,None,402325,0.00


In [46]:
%sql COPY point_data_centroid TO 'point_data_centroid.csv' (HEADER, DELIMITER ',');

Running query in 'duckdb:///local.db'

Count
